In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from functools import partial

from commons import *
from classifier import *

In [2]:
train = pd.DataFrame.from_csv(trainFile,sep=";",index_col=None)
test = pd.DataFrame.from_csv(testFile,sep=";",index_col=None)

In [3]:
train_t = pd.DataFrame.from_csv(trainTransformed,sep=";",index_col=None)
test_t = pd.DataFrame.from_csv(testTransformed,sep=";",index_col=None)

In [4]:
x_train = train

In [5]:
y_train = train[col_decision]

In [6]:
base_proba = {
    buy:0.33,
    hold:0.33,
    sell:0.33
}

confidence_map = {
    CONF_NO_EXPERT_MODEL:0.5,
    CONF_NO_EXPERT_SYMBOL_MODEL:0.75,
    CONF_ALL:1
}

In [7]:
clf = Classifier(base_proba,confidence_map)

In [8]:
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np
def Kcrossvalidation_transformed(clf, X, Y, X_transformed, K = 10):
    score = np.zeros(K)
    kf = StratifiedKFold(n_splits=K,shuffle=True)
    i = 0
    for train, test in kf.split(X,Y):
        print(i)
        X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], Y[train], Y[test]
        
        X_train_t = X_transformed[X_transformed[col_trans].isin(train)]
        X_test_t = X_transformed[X_transformed[col_trans].isin(test)]
        
        clf.fit_transformed(X_train_t)
        
        y_test_pred = clf.predict_transformed(X_test_t)
        print(set(y_test_pred))
        
        score[i] = accuracy(y_test,y_test_pred)
        i += 1
    return np.mean(score)


In [9]:
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np
def Kcrossvalidation(clf, X, Y, K = 10):
    score = np.zeros(K)
    kf = StratifiedKFold(n_splits=K,shuffle=True)
    i = 0
    for train, test in kf.split(X,Y):
        print(i)
        X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], Y[train], Y[test]
        clf.fit(X_train, y_train)
        
        y_test_pred = clf.predict(X_test)
        score[i] = accuracy(y_test,y_test_pred)
        i += 1
    return np.mean(score)

In [10]:
Kcrossvalidation_transformed(clf,x_train,y_train,train_t,3)

0


E:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
E:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
E:\workspace\git\ismis\classifier.py:102: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  byexpert = byexpert.sort(['TimeBefore'])
E:\workspace\git\ismis\classifier.py:43: RuntimeWarning: invalid value encountered in double_scalars
  proba[option] = ((1 - proba[pred]) * (1 - self.model.recall(option))) / s_recall


{'Sell', 'Hold', 'Buy'}
[[1070  538  479]
 [ 385  292  213]
 [ 453  305  345]]
1
{'Sell', 'Hold', 'Buy'}
[[1062  556  462]
 [ 370  284  232]
 [ 475  294  342]]
2
{'Sell', 'Hold', 'Buy'}
[[1132  557  489]
 [ 372  267  222]
 [ 403  310  325]]


0.50670208755707247